# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

In [ ]:
# Initialization

load_dotenv(override=True)

# openai_api_key = os.getenv('OPENAI_API_KEY')
# if openai_api_key:
#     print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
# else:
#     print("OpenAI API Key not set")
    
# MODEL = "gpt-4.1-mini"
# openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
MODEL = "llama3.2"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


In [ ]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

get_price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

set_price_function = {
    "name": "set_ticket_price",
    "description": "Set the price of a ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city that the price is being set for",
            },
            "price": {
                "type": "number",
                "description": "The price of the ticket in USD",
            },
        },
        "required": ["city", "price"],
        "additionalProperties": False
    }
}

get_all_prices_function = {
    "name": "get_all_ticket_prices",
    "description": "Get all the ticket prices.",
    "parameters": None
}


In [ ]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [ ]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [ ]:
def set_ticket_price(city, price):
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [ ]:
def get_all_ticket_prices():
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM prices')
        result = cursor.fetchall()
        return result

In [ ]:
# Define available functions in a dictionary for dynamic lookup
available_functions = {
    "get_ticket_price": get_ticket_price,
    "set_ticket_price": set_ticket_price,
    "get_all_ticket_prices": get_all_ticket_prices
}

def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions.get(function_name)
        
        if function_to_call:
            # Parse arguments from JSON
            arguments = json.loads(tool_call.function.arguments)
            
            try:
                result = function_to_call(**arguments)
                
                content = str(result) if result is not None else f"Function {function_name} executed successfully"
                    
            except Exception as e:
                content = f"Error executing {function_name}: {str(e)}"
            
            responses.append({
                "role": "tool",
                "content": content,
                "tool_call_id": tool_call.id
            })
        else:
            responses.append({
                "role": "tool",
                "content": f"Error: Function {function_name} not found",
                "tool_call_id": tool_call.id
            })
    
    return responses

def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

tools = [
    {"type": "function", "function": get_price_function}, 
    {"type": "function", "function": set_price_function},
    {"type": "function", "function": get_all_prices_function}
]

gr.ChatInterface(fn=chat, type="messages").launch()

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Hopefully this hardly needs to be stated! You now have the ability to give actions to your LLMs. This Airline Assistant can now do more than answer questions - it could interact with booking APIs to make bookings!</span>
        </td>
    </tr>
</table>